In [4]:
import keras
import numpy as np
from sklearn.metrics import classification_report
from sklearn.datasets import load_iris
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import StandardScaler

2024-03-24 12:02:08.312367: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 12:02:08.365124: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 12:02:08.365163: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 12:02:08.366947: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 12:02:08.375768: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 12:02:08.376156: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
X, y = load_iris(return_X_y=True)
xtrainF, xtestF, ytrainF, ytestF = train_test_split(X, y, random_state=42 ,test_size=0.20)
xtrainF.shape

(120, 4)

In [6]:
pesos = compute_class_weight(class_weight='balanced',classes=np.unique(ytrainF),y=ytrainF)
pesos = {
    0:pesos[0],
    1:pesos[1]
}

In [7]:
def Model():
    initalizaer = keras.initializers.VarianceScaling(scale=2., mode='fan_in', distribution='normal',seed=32)

    model = keras.models.Sequential(
        layers = [
            keras.layers.BatchNormalization(),
            keras.layers.Dense(8, activation=keras.activations.relu, kernel_initializer=initalizaer),
            keras.layers.BatchNormalization(),
            keras.layers.Dense(4, activation=keras.activations.relu, kernel_initializer=initalizaer),
            keras.layers.BatchNormalization(),
            keras.layers.Dense(3, activation=keras.activations.softmax)
        ]
    )
    return model

## Momentum

In [8]:
model = Model()
model.compile(
    optimizer = keras.optimizers.SGD(learning_rate=0.001,momentum=0.9),
    loss      = keras.losses.sparse_categorical_crossentropy,
    metrics   = [keras.metrics.sparse_categorical_accuracy]
)
call = keras.callbacks.EarlyStopping(monitor='loss', patience=5)
model.fit( xtrainF, ytrainF, epochs=32, batch_size=5, callbacks=[call])

Epoch 1/32
24/24 [==============================] - 1s 2ms/step - loss: 1.3691 - sparse_categorical_accuracy: 0.2583
Epoch 2/32
24/24 [==============================] - 0s 2ms/step - loss: 1.0282 - sparse_categorical_accuracy: 0.4583
Epoch 3/32
24/24 [==============================] - 0s 2ms/step - loss: 0.9466 - sparse_categorical_accuracy: 0.5667
Epoch 4/32
24/24 [==============================] - 0s 2ms/step - loss: 0.8897 - sparse_categorical_accuracy: 0.6083
Epoch 5/32
24/24 [==============================] - 0s 2ms/step - loss: 0.8850 - sparse_categorical_accuracy: 0.6417
Epoch 6/32
24/24 [==============================] - 0s 1ms/step - loss: 0.8456 - sparse_categorical_accuracy: 0.6583
Epoch 7/32
24/24 [==============================] - 0s 1ms/step - loss: 0.7799 - sparse_categorical_accuracy: 0.6833
Epoch 8/32
24/24 [==============================] - 0s 2ms/step - loss: 0.7680 - sparse_categorical_accuracy: 0.6833
Epoch 9/32
24/24 [==============================] - 0s 2ms/step 

In [9]:
pred = np.argmax(model.predict(xtestF),axis=-1)

1/1 [==============================] - 0s 235ms/step


In [10]:
print(classification_report(ytestF,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.90      1.00      0.95         9
           2       1.00      0.91      0.95        11

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



In [11]:
model.save('model/Model_iris0.keras')
model = keras.models.load_model('model/Model_iris0.keras')
model.fit( xtrainF, ytrainF, epochs=32, batch_size=5, verbose=0)

In [12]:
pred = np.argmax(model.predict(xtestF),axis=-1)

1/1 [==============================] - 0s 85ms/step


In [13]:
print(classification_report(ytestF,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



&nbsp;

## Nesterov

In [14]:
model = Model()
model.compile(
    optimizer = keras.optimizers.SGD(learning_rate=0.001,momentum=0.9, nesterov=True),
    loss      = keras.losses.sparse_categorical_crossentropy,
    metrics   = [keras.metrics.sparse_categorical_accuracy]
)
call = keras.callbacks.EarlyStopping(monitor='loss', patience=5)
model.fit( xtrainF, ytrainF, epochs=32, batch_size=5, callbacks=[call])

Epoch 1/32
24/24 [==============================] - 1s 2ms/step - loss: 1.5362 - sparse_categorical_accuracy: 0.3167 
Epoch 2/32
24/24 [==============================] - 0s 2ms/step - loss: 0.8885 - sparse_categorical_accuracy: 0.5917
Epoch 3/32
24/24 [==============================] - 0s 2ms/step - loss: 0.8036 - sparse_categorical_accuracy: 0.6250
Epoch 4/32
24/24 [==============================] - 0s 2ms/step - loss: 0.7580 - sparse_categorical_accuracy: 0.6500
Epoch 5/32
24/24 [==============================] - 0s 2ms/step - loss: 0.7217 - sparse_categorical_accuracy: 0.7083
Epoch 6/32
24/24 [==============================] - 0s 2ms/step - loss: 0.7457 - sparse_categorical_accuracy: 0.6833
Epoch 7/32
24/24 [==============================] - 0s 5ms/step - loss: 0.6594 - sparse_categorical_accuracy: 0.7583
Epoch 8/32
24/24 [==============================] - 0s 1ms/step - loss: 0.7585 - sparse_categorical_accuracy: 0.6750
Epoch 9/32
24/24 [==============================] - 0s 1ms/step

In [16]:
pred = np.argmax(model.predict(xtestF),axis=-1)

1/1 [==============================] - 0s 130ms/step


In [17]:
print(classification_report(ytestF,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.89      0.94         9
           2       0.92      1.00      0.96        11

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.97        30
weighted avg       0.97      0.97      0.97        30



### Adagrad

In [19]:
model = Model()
model.compile(
    optimizer = keras.optimizers.Adagrad(),
    loss      = keras.losses.sparse_categorical_crossentropy,
    metrics   = [keras.metrics.sparse_categorical_accuracy]
)
call = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
model.fit( xtrainF, ytrainF, epochs=32, batch_size=5, callbacks=[call])

Epoch 1/32


24/24 [==============================] - 1s 2ms/step - loss: 1.5334 - sparse_categorical_accuracy: 0.2917
Epoch 2/32
24/24 [==============================] - 0s 2ms/step - loss: 1.4777 - sparse_categorical_accuracy: 0.3333
Epoch 3/32
24/24 [==============================] - 0s 2ms/step - loss: 1.5179 - sparse_categorical_accuracy: 0.2917
Epoch 4/32
24/24 [==============================] - 0s 2ms/step - loss: 1.5077 - sparse_categorical_accuracy: 0.2750
Epoch 5/32
24/24 [==============================] - 0s 2ms/step - loss: 1.4288 - sparse_categorical_accuracy: 0.2667
Epoch 6/32
24/24 [==============================] - 0s 3ms/step - loss: 1.4018 - sparse_categorical_accuracy: 0.3333
Epoch 7/32
24/24 [==============================] - 0s 3ms/step - loss: 1.3895 - sparse_categorical_accuracy: 0.3583
Epoch 8/32
24/24 [==============================] - 0s 2ms/step - loss: 1.4158 - sparse_categorical_accuracy: 0.3083
Epoch 9/32
24/24 [==============================] - 0s 2ms/step - loss: 1.4

In [20]:
pred = np.argmax(model.predict(xtestF),axis=-1)

1/1 [==============================] - 0s 165ms/step


In [21]:
print(classification_report(ytestF,pred))

              precision    recall  f1-score   support

           0       0.33      0.10      0.15        10
           1       0.17      0.11      0.13         9
           2       0.38      0.73      0.50        11

    accuracy                           0.33        30
   macro avg       0.29      0.31      0.26        30
weighted avg       0.30      0.33      0.27        30



&nbsp;

### Adaline